In [ ]:
#テスト用

In [2]:
import streamlit as st

import pandas as pd

from PIL import Image

import os



# ワインセラーの設定

st.title('ワインセラー管理アプリ')



# ワインセラーの設定フォーム

with st.form("セラー設定フォーム"):

    rows = st.number_input("段数を設定", min_value=1, max_value=20, value=5)

    bottles_per_row = st.number_input("一段あたりの本数を設定", min_value=1, max_value=20, value=6)

    submit_button = st.form_submit_button(label="設定を保存")



# セラーの状態を保存

if submit_button:

    st.session_state.rows = rows

    st.session_state.bottles_per_row = bottles_per_row

    st.session_state.cell_count = rows * bottles_per_row

    st.session_state.wines = pd.DataFrame(columns=[

        'ワイン名', '年', '種類', '場所', '詳細情報', '写真', 

        '購入日', '価格', '購入場所', '国', '地域', '評価'

    ])

    st.session_state.wine_locations = {f"段{row+1} 本{bottle+1}": None for row in range(rows) for bottle in range(bottles_per_row)}

    st.success(f"セラーが設定されました！ 段数: {rows}, 一段あたりの本数: {bottles_per_row}")



# ワインの登録画面

st.subheader('ワイン登録')

wine_name = st.text_input("ワイン名")

wine_year = st.text_input("年")

wine_type = st.text_input("種類")

wine_location = st.selectbox("ワインの場所を選択", list(st.session_state.wine_locations.keys()))

wine_details = st.text_area("詳細情報")

wine_images = st.file_uploader("ワインの写真を最大3枚アップロード", type=['jpg', 'jpeg', 'png'], accept_multiple_files=True)

purchase_date = st.date_input("購入日")

price = st.number_input("価格", min_value=0.0, format="%.2f")

purchase_place = st.text_input("購入場所")

wine_country = st.text_input("ワインの国")

wine_region = st.text_input("ワインの地域")

wine_rating = st.slider("評価 (1〜5)", 1, 5, 3)



if st.button('ワインを登録'):

    if wine_name and wine_year and wine_type and wine_location and wine_details:

        # 最大3枚の写真をアップロード

        if wine_images:

            image_paths = []

            for i, wine_image in enumerate(wine_images[:3]):  # 最大3枚まで処理

                image = Image.open(wine_image)

                image_path = f"images/{wine_name}_{wine_location}_photo_{i+1}.png"

                os.makedirs(os.path.dirname(image_path), exist_ok=True)

                image.save(image_path)

                image_paths.append(image_path)

        else:

            image_paths = []

        

        # 登録したワイン情報をDataFrameに追加

        wine_info = {

            'ワイン名': wine_name, 

            '年': wine_year, 

            '種類': wine_type, 

            '場所': wine_location, 

            '詳細情報': wine_details, 

            '写真': image_paths, 

            '購入日': purchase_date,

            '価格': price, 

            '購入場所': purchase_place, 

            '国': wine_country,

            '地域': wine_region,

            '評価': wine_rating

        }

        st.session_state.wines = st.session_state.wines.append(wine_info, ignore_index=True)

        st.session_state.wine_locations[wine_location] = wine_name

        st.success(f"{wine_name}が登録されました。")

    else:

        st.error("すべてのフィールドを入力してください。")



# セラーの表示

st.subheader('ワインセラーの状態')



# ワインセラーのレイアウト表示

wine_cells = st.session_state.wine_locations

for row in range(st.session_state.rows):

    row_label = f"段{row+1}"

    row_cells = []

    for bottle in range(st.session_state.bottles_per_row):

        location = f"{row_label} 本{bottle+1}"

        wine_name_at_location = wine_cells.get(location, "空")

        if wine_name_at_location == "空":

            row_cells.append("空")

        else:

            # ワイン名と写真を表示

            wine_info = st.session_state.wines[st.session_state.wines['ワイン名'] == wine_name_at_location].iloc[0]

            image_paths = wine_info['写真']

            if image_paths:

                img = Image.open(image_paths[0])  # 1枚目の写真を表示

                row_cells.append(f"{wine_name_at_location} : {img}")

            else:

                row_cells.append(f"{wine_name_at_location}")

    st.write(f"{row_label}: " + " | ".join(row_cells))



# ワイン詳細情報表示

st.subheader('ワインの詳細情報')



# 検索ボックスでワインを選ぶ

wine_selection = st.selectbox('ワインを選んで詳細情報を確認', st.session_state.wines['ワイン名'].unique())



if wine_selection:

    selected_wine = st.session_state.wines[st.session_state.wines['ワイン名'] == wine_selection].iloc[0]

    st.write(f"ワイン名: {selected_wine['ワイン名']}")

    st.write(f"年: {selected_wine['年']}")

    st.write(f"種類: {selected_wine['種類']}")

    st.write(f"場所: {selected_wine['場所']}")

    st.write(f"詳細情報: {selected_wine['詳細情報']}")

    st.write(f"購入日: {selected_wine['購入日']}")

    st.write(f"価格: {selected_wine['価格']}円")

    st.write(f"購入場所: {selected_wine['購入場所']}")

    st.write(f"国: {selected_wine['国']}")

    st.write(f"地域: {selected_wine['地域']}")

    st.write(f"評価: {selected_wine['評価']}点")

    if selected_wine['写真']:

        for idx, photo in enumerate(selected_wine['写真']):

            st.image(photo, caption=f"ワインの写真 {idx + 1}", use_column_width=True)



# ワイン編集機能

st.subheader('ワイン情報の編集')



wine_to_edit = st.selectbox('編集するワインを選択', st.session_state.wines['ワイン名'].unique())



if wine_to_edit:

    selected_wine_for_edit = st.session_state.wines[st.session_state.wines['ワイン名'] == wine_to_edit].iloc[0]

    

    edited_wine_name = st.text_input("ワイン名", value=selected_wine_for_edit['ワイン名'])

    edited_wine_year = st.text_input("年", value=selected_wine_for_edit['年'])

    edited_wine_type = st.text_input("種類", value=selected_wine_for_edit['種類'])

    edited_wine_details = st.text_area("詳細情報", value=selected_wine_for_edit['詳細情報'])

    edited_wine_image = st.file_uploader("ワインの写真をアップロード (最大3枚)", type=['jpg', 'jpeg', 'png'], accept_multiple_files=True, key="edit_image")

    edited_purchase_date = st.date_input("購入日", value=selected_wine_for_edit['購入日'])

    edited_price = st.number_input("価格", value=selected_wine_for_edit['価格'], min_value=0.0, format="%.2f")

    edited_purchase_place = st.text_input("購入場所", value=selected_wine_for_edit['購入場所'])

    edited_wine_country = st.text_input("ワインの国", value=selected_wine_for_edit['国'])

    edited_wine_region = st.text_input("ワインの地域", value=selected_wine_for_edit['地域'])

    edited_wine_rating = st.slider("評価 (1〜5)", 1, 5, selected_wine_for_edit['評価'])



    if st.button('編集を保存'):

        if edited_wine_name and edited_wine_year and edited_wine_type and edited_wine_details:

            # 新しい画像のアップロード

            if edited_wine_image is not None:

                image_paths = []

                for i, wine_image in enumerate(edited_wine_image[:3]):

                    image = Image.open(wine_image)

                    image_path = f"images/{edited_wine_name}_{selected_wine_for_edit['場所']}_photo_{i+1}.png"

                    os.makedirs(os.path.dirname(image_path), exist_ok=True)

                    image.save(image_path)

                    image_paths.append(image_path)

            else:

                image_paths = selected_wine_for_edit['写真']



            # 更新したワイン情報をDataFrameに保存

            st.session_state.wines.loc[st.session_state.wines['ワイン名'] == wine_to_edit, [

                'ワイン名', '年', '種類', '詳細情報', '写真', '購入日', '価格', '購入場所', '国', '地域', '評価']] = [

                edited_wine_name, edited_wine_year, edited_wine_type, edited_wine_details, image_paths, 

                edited_purchase_date, edited_price, edited_purchase_place, edited_wine_country, 

                edited_wine_region, edited_wine_rating]

            st.success(f"{edited_wine_name}の情報が更新されました。")

        else:

            st.error("すべてのフィールドを入力してください。")


2025-02-15 16:19:57.303 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:19:57.528 
  command:

    streamlit run c:\Users\Yoshimi\Anaconda3\envs\pymc_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-15 16:19:57.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:19:57.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:19:57.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:19:57.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:19:57.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:19:57.540 Thread 

AttributeError: st.session_state has no attribute "wine_locations". Did you forget to initialize it? More info: https://docs.streamlit.io/develop/concepts/architecture/session-state#initialization